# Data Extra, and Transform

In [5]:
import pandas as pd
import collections
import sqlite3
import sqlalchemy
from IPython.display import display

## Use the 2018 dataset as the base case for all other years.

- We only need to make a restaurant table and food category table once from the 2018 dataset
- For all other years, we only need to make the item_info table, nutrition_facts table, and the combo_facts table

In [6]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2018.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,35,36,37,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Restaurant dimension

In [7]:
dat['r_id'] = dat.groupby(['Restaurant']).ngroup()
restaurant = dat[['r_id', 'Restaurant']].drop_duplicates().sort_values(by=['r_id'])
restaurant.rename(columns={'Restaurant':'r_name'}, inplace=True)
restaurant
#restaurant.to_csv('cleaned_tables_by_year/2018/restaurant.csv', index=False)

,r_id,r_name
2120,0,7 Eleven
4,1,Applebee's
1153,2,Arby's
4345,3,Auntie Anne's
1817,4,BJ's Restaurant & Brewhouse
...,...,...
1155,91,Whataburger
1482,92,White Castle
1524,93,Wingstop
1537,94,Yard House


### Food category dimension

In [8]:
dat['cat_id'] = dat.groupby(['Food_Category']).ngroup()
food_category = dat[['cat_id', 'Food_Category']].drop_duplicates().sort_values(by=['cat_id'])
food_category.rename(columns={'Food_Category':'cat_name'}, inplace=True)
food_category
#food_category.to_csv('cleaned_tables_by_year/2018/food_category.csv', index=False)


,cat_id,cat_name
12,0,Appetizers & Sides
369,1,Baked Goods
16,2,Beverages
130,3,Burgers
263,4,Desserts
0,5,Entrees
132,6,Fried Potatoes
306,7,Pizza
32,8,Salads
1,9,Sandwiches


### Item information dimension

In [9]:
item_info = dat[['Menu_Item_ID', 'Year', 'Item_Name', 'Item_Description']].drop_duplicates()
item_info.rename(columns={'Menu_Item_ID':'item_id', 'Year':'year', 'Item_Name':'item_name', 'Item_Description':'item_description'}, inplace=True)
#item_info.to_csv('cleaned_tables_by_year/2018/item_info.csv', index=False)
item_info

,item_id,year,item_name,item_description
0,100054,2018,Burrito Bowl,"Burrito Bowl, Burritos & More, Vegetarian Eats..."
1,100057,2018,Quesadilla Kids Meal,"Quesadilla Kids Meal w/ Quesadilla, Applesauce..."
2,100060,2018,Burrito,"Burrito, Burritos & More, Vegetarian Eats; Cho..."
3,100073,2018,"Flour Tortilla, 12.5 in","Flour Tortilla, 12.5 in, Ingredients for Entrees"
4,10012,2018,French Onion Soup,"French Onion Soup w/ Melted Cheese & Croutons,..."
...,...,...,...,...
71167,99835,2018,"Buffalo Chicken Pizza on Hand Tossed Crust, La...","Buffalo Chicken Pizza w/ Buffalo Pizza Sauce, ..."
71168,99966,2018,Tenders Family Meals,"Tenders Family Meals w/ Handcrafted Tenders, S..."
71169,99968,2018,3 Cajun Fish Filets,"3 Cajun Fish Filets, Seafood, Not Available At..."
71170,99969,2018,Fish & Popcorn Shrimp,"Fish & Popcorn Shrimp w/ Side & Biscuit, Seafood"


### Nutrition fact table

In [10]:
# Fact table, nutrition data
nutrition_facts = dat[['Menu_Item_ID', 'Year', 'r_id', 'cat_id',
                       'Calories', 'Total_Fat', 'Saturated_Fat', 'Trans_Fat', 'Cholesterol',
                       'Sodium', 'Potassium',
                       'Carbohydrates', 'Protein', 'Sugar', 'Dietary_Fiber']]
nutrition_facts.columns = ['item_id', 'year', 'r_id', 'cat_id',
                           'calories', 'total_fat', 'saturated_fat', 'trans_fat', 'cholesterol',
                           'sodium', 'potassium',
                           'carbs', 'protein', 'sugar', 'dietary_Fiber']
nutrition_facts = nutrition_facts.drop_duplicates()
#nutrition_facts.astype({'calories':'int64', 'total_fat':'int64', 'saturated_fat':'int64', 'trans_fat':'int64',
#                        'cholesterol':'int64', 'sodium':'int64', 'potassium':'int64',
#                           'carbs':'int64', 'protein':'int64', 'sugar':'int64', 'dietary_Fiber':'int64'})
nutrition_facts
#nutrition_facts.to_csv('cleaned_tables_by_year/2018/nutrition_facts.csv', index=False)

,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100054,2018,72,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100057,2018,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100060,2018,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100073,2018,72,11,300.0,7.0,3.0,0.0,0.0,760.0,50.0,52.0,8.0,2.0,3.0
4,10012,2018,1,10,380.0,22.0,13.0,0.0,60.0,1390.0,NaN,27.0,17.0,7.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71167,99835,2018,69,7,270.0,8.0,4.0,0.0,30.0,750.0,NaN,37.0,13.0,1.0,2.0
71168,99966,2018,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71169,99968,2018,70,5,380.0,19.0,5.0,0.0,40.0,1650.0,NaN,37.0,16.0,4.0,2.0
71170,99969,2018,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Combo builds fact table

In [7]:
# Combo builds fact table
combo_facts = dat[dat['Customizable_Build_ID'].notna()]
combo_facts = combo_facts[['Customizable_Build_ID', 'Menu_Item_ID', 'Year', 'Customizable_Builds', 'cat_id']]
combo_facts.columns = ['combo_id', 'item_id', 'year', 'builds', 'cat_id']
combo_facts = combo_facts.astype({'combo_id':'int64'})
#combo_facts.to_csv('cleaned_tables_by_year/2018/combo_facts.csv', index=False)
combo_facts

,combo_id,item_id,year,builds,cat_id
0,704214541,100054,2018,Main Item,5
1,70814571,100057,2018,Main Item,9
2,704214551,100060,2018,Main Item,9
5,2301341,10012,2018,Accompanying Item,10
6,2331351,10012,2018,Accompanying Item,10
...,...,...,...,...,...
71146,66913561,99683,2018,Accompanying Item,8
71147,66913631,99683,2018,Accompanying Item,8
71148,661513201,99684,2018,Main Item,8
71152,81818032,997,2018,Main Item,9


In [8]:
### testing for combo combinations, might be too complicated to implement
h = collections.defaultdict(list)
combo = combo_facts[combo_facts['combo_id'] == 2331351]
key = 1
for i in combo['cat_id'].unique():
    h[key] = combo[combo['cat_id'] == i]['item_id'].to_list()
    key += 1
if max(h.keys()) == 5:
    combin = [(a, b, c, d, e) for a in h[1] for b in h[2] for c in h[3] for d in h[4] for e in h[5]]
    #print(combin)

## Testing for 2017 data

In [8]:
dat2017 = pd.read_csv('Customizable_Annual_Data/Annual_Data2017.csv')

# Get restaurant ID
dat2017 = pd.merge(dat2017, restaurant, left_on=['Restaurant'], right_on='r_name', how='left')
# Get food category ID
dat2017 = pd.merge(dat2017, food_category, left_on=['Food_Category'], right_on='cat_name', how='left')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,36,37,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
item_info_2017 = dat2017[['Menu_Item_ID', 'Year', 'Item_Name', 'Item_Description']].drop_duplicates()
item_info_2017.rename(columns={'Menu_Item_ID':'item_id', 'Year':'year', 'Item_Name':'item_name', 'Item_Description':'item_description'}, inplace=True)
item_info_2017

,item_id,year,item_name,item_description
0,100054,2017,Burrito Bowl,Burrito Bowl; Choice of 1 Protein: Grilled Ado...
1,100057,2017,Quesadilla Kids Meal,"Quesadilla Kids Meal w/ Quesadilla, Applesauce..."
2,100060,2017,Burrito,"Burrito; Choice of 1 Protein: Grilled Adobo, T..."
3,100073,2017,Flour Tortilla,"Flour Tortilla, 12.5 in, Ingredients for Entrees"
5,100078,2017,Kids Meal,Kids Meal w/ Entree & Small Fountain Drink; Ch...
...,...,...,...,...
87808,99965,2017,Seafood Combo,"Seafood Combo w/ Signature Side, Biscuit & Dri..."
87809,99966,2017,Chicken Tenders Family Meals,"Chicken Tenders Family Meals, Popeyes Handcraf..."
87810,99968,2017,Cajun Fish,"Cajun Fish, Seafood"
87812,99969,2017,Fish & Popcorn Shrimp,"Fish & Popcorn Shrimp, Seafood"


In [11]:
# Fact table, nutrition data
nutrition_facts_2017 = dat2017[['Menu_Item_ID', 'Year', 'r_id', 'cat_id',
                       'Calories', 'Total_Fat', 'Saturated_Fat', 'Trans_Fat', 'Cholesterol',
                       'Sodium', 'Potassium',
                       'Carbohydrates', 'Protein', 'Sugar', 'Dietary_Fiber']]
nutrition_facts_2017.columns = ['item_id', 'year', 'r_id', 'cat_id',
                           'calories', 'total_fat', 'saturated_fat', 'trans_fat', 'cholesterol',
                           'sodium', 'potassium',
                           'carbs', 'protein', 'sugar', 'dietary_Fiber']
nutrition_facts_2017 = nutrition_facts_2017.drop_duplicates()
nutrition_facts_2017

,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100054,2017,72,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100057,2017,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100060,2017,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100073,2017,72,11,300.0,7.0,3.0,0.0,0.0,760.0,50.0,52.0,8.0,2.0,3.0
5,100078,2017,73,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87808,99965,2017,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87809,99966,2017,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87810,99968,2017,70,5,130.0,2.0,0.0,0.0,15.0,550.0,NaN,12.0,5.0,1.0,1.0
87812,99969,2017,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Combo builds fact table
combo_facts_2017 = dat2017[dat2017['Customizable_Build_ID'].notna()]
combo_facts_2017 = combo_facts_2017[['Customizable_Build_ID', 'Menu_Item_ID', 'Year', 'Customizable_Builds', 'cat_id']]
combo_facts_2017.columns = ['combo_id', 'item_id', 'year', 'builds', 'cat_id']
combo_facts_2017 = combo_facts_2017.astype({'combo_id':'int64'})
combo_facts_2017

,combo_id,item_id,year,builds,cat_id
0,701119011,100054,2017,Main Item,5
1,702119031,100057,2017,Main Item,9
2,701119001,100060,2017,Main Item,9
4,705019061,100073,2017,Accompanying Item,11
5,712519131,100078,2017,Main Item,5
...,...,...,...,...,...
87776,81222381,997,2017,Main Item,9
87807,692818991,99964,2017,Main Item,5
87808,692718981,99965,2017,Main Item,5
87809,692818961,99966,2017,Main Item,5


In [13]:
item_info_2017.to_csv('cleaned_tables_by_year/2017/item_info.csv', index=False)
nutrition_facts_2017.to_csv('cleaned_tables_by_year/2017/nutrition_facts.csv', index=False)
combo_facts_2017.to_csv('cleaned_tables_by_year/2017/combo_facts.csv', index=False)


## Use the following function to transform the rest of the datasets

In [122]:
### This function is use to transform data for files other then 2018
def transform_data(dat, year):
    # Get restaurant ID
    dat = pd.merge(dat, restaurant, left_on=['Restaurant'], right_on='r_name', how='left')
    # Get food category ID
    dat = pd.merge(dat, food_category, left_on=['Food_Category'], right_on='cat_name', how='left')

    item_info = dat[['Menu_Item_ID', 'Year', 'Item_Name', 'Item_Description']].drop_duplicates()
    item_info.rename(columns={'Menu_Item_ID':'item_id', 'Year':'year', 'Item_Name':'item_name', 'Item_Description':'item_description'}, inplace=True)
    display(item_info)

    # Check na
    print(item_info['item_id'].isnull().sum())
    print(item_info['year'].isnull().sum())
    print(item_info['item_name'].isnull().sum())

    # Fact table, nutrition data
    nutrition_facts = dat[['Menu_Item_ID', 'Year', 'r_id', 'cat_id',
                           'Calories', 'Total_Fat', 'Saturated_Fat', 'Trans_Fat', 'Cholesterol',
                           'Sodium', 'Potassium',
                           'Carbohydrates', 'Protein', 'Sugar', 'Dietary_Fiber']]
    nutrition_facts.columns = ['item_id', 'year', 'r_id', 'cat_id',
                               'calories', 'total_fat', 'saturated_fat', 'trans_fat', 'cholesterol',
                               'sodium', 'potassium',
                               'carbs', 'protein', 'sugar', 'dietary_Fiber']
    nutrition_facts = nutrition_facts.drop_duplicates()
    display(nutrition_facts)

    # Check na
    print(nutrition_facts['item_id'].isnull().sum())
    print(nutrition_facts['year'].isnull().sum())
    print(nutrition_facts['r_id'].isnull().sum())
    print(nutrition_facts['cat_id'].isnull().sum())

    # Combo builds fact table
    combo_facts = dat[dat['Customizable_Build_ID'].notna()]
    combo_facts = combo_facts[['Customizable_Build_ID', 'Menu_Item_ID', 'Year', 'Customizable_Builds', 'cat_id']]
    combo_facts.columns = ['combo_id', 'item_id', 'year', 'builds', 'cat_id']
    combo_facts = combo_facts.astype({'combo_id':'int64'})
    display(combo_facts)

    # Check na
    print(combo_facts['combo_id'].isnull().sum())
    print(combo_facts['item_id'].isnull().sum())
    print(combo_facts['year'].isnull().sum())
    print(combo_facts['cat_id'].isnull().sum())
    print(combo_facts['builds'].isnull().sum())

    file_path = 'cleaned_tables_by_year/' + year +'/'
    item_info.to_csv(file_path + 'item_info.csv', index=False)
    nutrition_facts.to_csv(file_path + 'nutrition_facts.csv', index=False)
    combo_facts.to_csv(file_path + 'combo_facts.csv', index=False)



In [123]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2017.csv')
transform_data(dat, year='2017')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,36,37,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,item_id,year,item_name,item_description
0,100054,2017,Burrito Bowl,Burrito Bowl; Choice of 1 Protein: Grilled Ado...
1,100057,2017,Quesadilla Kids Meal,"Quesadilla Kids Meal w/ Quesadilla, Applesauce..."
2,100060,2017,Burrito,"Burrito; Choice of 1 Protein: Grilled Adobo, T..."
3,100073,2017,Flour Tortilla,"Flour Tortilla, 12.5 in, Ingredients for Entrees"
5,100078,2017,Kids Meal,Kids Meal w/ Entree & Small Fountain Drink; Ch...
...,...,...,...,...
87808,99965,2017,Seafood Combo,"Seafood Combo w/ Signature Side, Biscuit & Dri..."
87809,99966,2017,Chicken Tenders Family Meals,"Chicken Tenders Family Meals, Popeyes Handcraf..."
87810,99968,2017,Cajun Fish,"Cajun Fish, Seafood"
87812,99969,2017,Fish & Popcorn Shrimp,"Fish & Popcorn Shrimp, Seafood"


0
0
0
0
0
0
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100054,2017,72,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100057,2017,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100060,2017,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100073,2017,72,11,300.0,7.0,3.0,0.0,0.0,760.0,50.0,52.0,8.0,2.0,3.0
5,100078,2017,73,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87808,99965,2017,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87809,99966,2017,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87810,99968,2017,70,5,130.0,2.0,0.0,0.0,15.0,550.0,NaN,12.0,5.0,1.0,1.0
87812,99969,2017,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,combo_id,item_id,year,builds,cat_id
0,701119011,100054,2017,Main Item,5
1,702119031,100057,2017,Main Item,9
2,701119001,100060,2017,Main Item,9
4,705019061,100073,2017,Accompanying Item,11
5,712519131,100078,2017,Main Item,5
...,...,...,...,...,...
87776,81222381,997,2017,Main Item,9
87807,692818991,99964,2017,Main Item,5
87808,692718981,99965,2017,Main Item,5
87809,692818961,99966,2017,Main Item,5


In [131]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2016.csv')
transform_data(dat, year='2016')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (37,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,item_id,year,item_name,item_description
0,10004,2016,"Tomato Basil Soup, for Pick 'n Pair & Lunch Co...","Tomato Basil Soup, for Pick 'n Pair & Lunch Co..."
2,100052,2016,Lil' Chicken Nachos Kids Meal,Lil' Chicken Nachos Kids Meal w/ Handmade Tort...
3,100054,2016,Burrito Bowl,Burrito Bowl
4,100057,2016,Grilled Quesadilla Kids Meal,Grilled Quesadilla Kids Meal w/ Flour Tortilla...
5,100060,2016,Burrito,Burrito
...,...,...,...,...
69147,99965,2016,Seafood Combo,"Seafood Combo w/ Signature Side, Biscuit & Dri..."
69148,99966,2016,Chicken Tenders Family Meals,"Chicken Tenders Family Meals, Popeyes Handcraf..."
69149,99968,2016,Cajun Fish,"Cajun Fish, Seafood"
69150,99969,2016,Fish & Popcorn Shrimp,"Fish & Popcorn Shrimp, Seafood"


0
0
0
0
0
0
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,10004,2016,1,10,240.0,15.0,7.0,0.0,30.0,1090.0,NaN,20.0,6.0,12.0,3.0
2,100052,2016,72,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100054,2016,72,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100057,2016,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,100060,2016,72,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69147,99965,2016,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69148,99966,2016,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69149,99968,2016,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69150,99969,2016,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,combo_id,item_id,year,builds,cat_id
1,2251061,10004,2016,Accompanying Item,10
2,70153701,100052,2016,Main Item,0
3,70343641,100054,2016,Main Item,5
4,70183731,100057,2016,Main Item,9
5,70373661,100060,2016,Main Item,9
...,...,...,...,...,...
69118,81124233,998,2016,Main Item,9
69146,69433811,99964,2016,Main Item,5
69147,69393801,99965,2016,Main Item,5
69148,69433812,99966,2016,Main Item,5


In [132]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2015.csv')
transform_data(dat, year='2015')

,item_id,year,item_name,item_description
0,10001,2015,Toasted Breadstick,"Toasted Breadstick, Lunch-Sized Portion, Lunch..."
1,10002,2015,Toasted Garlic Breadstick Basket,"Toasted Garlic Breadstick Basket, Soups, Salad..."
27,10004,2015,"Tomato Basil Soup, Lunch-Sized Portion","Tomato Basil Soup, Lunch-Sized Portion, Lunch ..."
29,100052,2015,Lil' Chicken Nachos Kids Meal,Lil' Chicken Nachos Kids Meal w/ Handmade Tort...
30,100053,2015,Craft 2 Naked Burrito,"Craft 2 Naked Burrito; Choice of: Rice, Beans,..."
...,...,...,...,...
61103,99967,2015,1/4 LB Popcorn Shrimp,"1/4 LB Popcorn Shrimp, Seafood"
61105,99968,2015,Cajun Fish,"Cajun Fish, Seafood"
61107,99969,2015,Fish & Popcorn Shrimp,"Fish & Popcorn Shrimp, Seafood"
61109,99970,2015,Premium Butterfly Shrimp,"Premium Butterfly Shrimp, Seafood"


0
0
0
0
0
0
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,10001,2015,1,1,130.0,4.0,1.0,0.0,0.0,200.0,NaN,19.0,4.0,3.0,1.0
1,10002,2015,1,1,500.0,15.0,3.5,0.0,10.0,820.0,NaN,77.0,16.0,11.0,3.0
27,10004,2015,1,10,240.0,15.0,7.0,0.0,30.0,1090.0,NaN,20.0,6.0,12.0,3.0
29,100052,2015,72,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,100053,2015,72,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61103,99967,2015,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61105,99968,2015,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61107,99969,2015,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61109,99970,2015,70,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,combo_id,item_id,year,builds,cat_id
2,10331121,10002,2015,Accompanying Item,1
3,103361131,10002,2015,Accompanying Item,1
4,1033611310,10002,2015,Accompanying Item,1
5,1033611311,10002,2015,Accompanying Item,1
6,1033611312,10002,2015,Accompanying Item,1
...,...,...,...,...,...
61102,2131711187,99966,2015,Main Item,5
61104,2131711186,99967,2015,Accompanying Item,5
61106,2131711186,99968,2015,Accompanying Item,5
61108,2131711186,99969,2015,Accompanying Item,5


In [133]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2014.csv')
transform_data(dat, year='2014')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34,35,36,37,38,39,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,item_id,year,item_name,item_description
0,10001,2014,Breadstick,"Breadstick, Lunch Combos"
1,10002,2014,Toasted Garlic Bread Basket,"Toasted Garlic Bread Basket, Extras"
2,10004,2014,"Tomato Basil Soup, Lunch Combos","Tomato Basil Soup, Lunch Combos"
3,10005,2014,"Chili, Lunch Combos","Chili, Lunch Combos"
4,10006,2014,"Baked Potato Soup, Lunch Combos","Baked Potato Soup, Lunch Combos"
...,...,...,...,...
53304,989,2014,Breakfast Toaster w/ Sausage,"Breakfast Toaster w/ Melty Cheese, Savory Saus..."
53305,995,2014,Sausage & Cheddar Classic Breakfast Sandwich,"Sausage & Cheddar Classic Breakfast Sandwich, ..."
53306,996,2014,Bagel Sandwich w/ Sausage,"Bagel Sandwich on Grilled Bagel w/ Sausage, Eg..."
53307,997,2014,Bagel Sandwich w/ Bacon,"Bagel Sandwich w/ Bacon w/out Hash Browns, Bre..."


0
0
0
0
0
0
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,10001,2014,1,1,130.0,4.0,0.5,0.0,NaN,200.0,NaN,19.0,4.0,NaN,1.0
1,10002,2014,1,1,500.0,15.0,2.5,0.0,NaN,820.0,NaN,76.0,16.0,NaN,3.0
2,10004,2014,1,10,270.0,17.0,8.0,0.0,NaN,1440.0,NaN,25.0,6.0,NaN,3.0
3,10005,2014,1,10,370.0,23.0,12.0,0.5,NaN,1020.0,NaN,13.0,29.0,NaN,4.0
4,10006,2014,1,10,440.0,32.0,15.0,1.0,NaN,610.0,NaN,26.0,13.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53304,989,2014,81,9,670.0,39.0,13.0,1.0,340.0,1510.0,NaN,52.0,27.0,7.0,2.0
53305,995,2014,82,9,500.0,28.0,9.0,0.0,165.0,920.0,NaN,41.0,19.0,3.0,1.0
53306,996,2014,83,9,570.0,29.0,11.0,0.0,275.0,1240.0,NaN,50.0,26.0,4.0,2.0
53307,997,2014,83,9,450.0,16.0,7.0,0.0,275.0,1200.0,NaN,51.0,24.0,5.0,2.0


,combo_id,item_id,year,builds,cat_id
17,103211011,10020,2014,Main Item,8
25,103211012,10042,2014,Main Item,8
26,103211013,10043,2014,Main Item,8
29,103211014,10050,2014,Main Item,8
30,103211015,10051,2014,Main Item,8
...,...,...,...,...,...
53281,173213243,959,2014,Main Item,9
53283,173213244,960,2014,Main Item,9
53285,173213245,962,2014,Main Item,9
53306,22175711,996,2014,Main Item,9


In [134]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2013.csv')
transform_data(dat, year='2013')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,item_id,year,item_name,item_description
0,100,2013,"ButterBurger w/ Cheese, Kids","ButterBurger w/ Cheese, Kids"
1,10001,2013,Breadstick,"Breadstick, Pick 'n Pair & Lunch Combos"
2,10002,2013,Toasted Garlic Bread Basket,"Toasted Garlic Bread Basket, Extras, As Served"
3,10003,2013,"French Onion Soup, Pick 'n Pair & Lunch Combos","French Onion Soup, Pick 'n Pair & Lunch Combos"
4,10004,2013,"Tomato Basil Soup, Pick 'n Pair & Lunch Combos","Tomato Basil Soup, Pick 'n Pair & Lunch Combos"
...,...,...,...,...
38866,99,2013,"ButterBurger, Kids","ButterBurger, Kids"
38867,995,2013,Sausage & Cheddar Classic Breakfast Sandwich,"Sausage & Cheddar Classic Breakfast Sandwich, ..."
38868,996,2013,Sausage Bagel Sandwich,"Sausage Bagel Sandwich w/out Hash Browns, Brea..."
38869,997,2013,Bagel Sandwich w/ Bacon,"Bagel Sandwich w/ Bacon w/out Hash Browns, Bre..."


0
0
0
0
0
0
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100,2013,23,3,421.0,21.0,9.0,0.0,80.0,990.0,NaN,37.0,22.0,9.0,1.0
1,10001,2013,1,1,130.0,4.0,0.5,0.0,NaN,200.0,NaN,19.0,4.0,NaN,1.0
2,10002,2013,1,1,500.0,15.0,2.5,0.0,NaN,820.0,NaN,76.0,16.0,NaN,3.0
3,10003,2013,1,10,330.0,20.0,12.0,0.5,NaN,1230.0,NaN,22.0,15.0,NaN,1.0
4,10004,2013,1,10,220.0,14.0,6.0,0.0,NaN,1230.0,NaN,23.0,5.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38866,99,2013,23,3,331.0,15.0,6.0,0.0,65.0,440.0,NaN,31.0,19.0,4.0,1.0
38867,995,2013,82,9,500.0,28.0,9.0,0.0,165.0,920.0,NaN,41.0,19.0,3.0,1.0
38868,996,2013,83,9,570.0,29.0,11.0,0.0,250.0,1240.0,NaN,50.0,26.0,4.0,2.0
38869,997,2013,83,9,450.0,16.0,7.0,0.0,250.0,1200.0,NaN,51.0,24.0,5.0,2.0


,combo_id,item_id,year,builds,cat_id
14,103111031,10013,2013,Accompanying Item,10
15,103111032,10013,2013,Accompanying Item,10
16,103111033,10013,2013,Accompanying Item,10
17,103111034,10013,2013,Accompanying Item,10
18,103111035,10013,2013,Accompanying Item,10
...,...,...,...,...,...
38767,2332544610,819,2013,Main Item,9
38775,2223239519,833,2013,Main Item,9
38776,2223239520,834,2013,Main Item,9
38868,22163891,996,2013,Main Item,9


In [135]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2012.csv')
transform_data(dat, year='2012')


,item_id,year,item_name,item_description
0,100,2012,"ButterBurger w/ Cheese, Kids","ButterBurger w/ Cheese, Kids"
1,10001,2012,Breadstick,"Breadstick, Pick 'n Pair & Lunch Combos"
2,10002,2012,Toasted Garlic Bread Basket,"Toasted Garlic Bread Basket, Extras, As Served"
3,10003,2012,"French Onion Soup, Pick 'n Pair & Lunch Combos","French Onion Soup, Pick 'n Pair & Lunch Combos"
4,10004,2012,"Tomato Basil Soup, Pick 'n Pair & Lunch Combos","Tomato Basil Soup, Pick 'n Pair & Lunch Combos"
...,...,...,...,...
23703,99,2012,"ButterBurger, Kids","ButterBurger, Kids"
23704,995,2012,Sausage & Cheddar Classic Breakfast Sandwich,"Sausage & Cheddar Classic Breakfast Sandwich, ..."
23705,996,2012,Bagel Sandwich w/ Sausage,"Bagel Sandwich w/ Sausage, Breakfast"
23706,997,2012,Bagel Sandwich w/ Bacon,"Bagel Sandwich w/ Bacon, Breakfast"


0
0
0
0
0
229
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100,2012,23.0,3,421.0,22.0,10.0,1.0,80.0,990.0,NaN,37.0,22.0,9.0,1.0
1,10001,2012,1.0,1,280.0,19.0,6.0,0.0,NaN,330.0,NaN,22.0,5.0,NaN,1.0
2,10002,2012,1.0,1,1120.0,78.0,23.0,15.0,NaN,1320.0,NaN,89.0,19.0,NaN,6.0
3,10003,2012,1.0,10,240.0,14.0,9.0,0.0,NaN,1070.0,NaN,11.0,14.0,NaN,1.0
4,10004,2012,1.0,10,220.0,14.0,6.0,0.0,NaN,1220.0,NaN,23.0,5.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,99,2012,23.0,3,331.0,16.0,6.0,1.0,65.0,440.0,NaN,31.0,19.0,4.0,1.0
23704,995,2012,82.0,9,500.0,28.0,9.0,0.0,165.0,920.0,NaN,41.0,19.0,3.0,NaN
23705,996,2012,83.0,9,570.0,29.0,11.0,0.0,275.0,1240.0,NaN,50.0,26.0,4.0,2.0
23706,997,2012,83.0,9,450.0,16.0,7.0,0.0,275.0,1200.0,NaN,51.0,24.0,5.0,2.0


,combo_id,item_id,year,builds,cat_id
20,103271011,10020,2012,Main Item,8
34,103271012,10034,2012,Main Item,8
35,103271013,10035,2012,Main Item,8
38,103271014,10038,2012,Main Item,8
39,103271015,10039,2012,Main Item,8
...,...,...,...,...,...
23546,11431291,730,2012,Accompanying Item,11
23547,11431292,730,2012,Accompanying Item,11
23562,2223129816,758,2012,Main Item,9
23602,2332731111,819,2012,Main Item,9


In [136]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2010.csv')
transform_data(dat, year='2010')

,item_id,year,item_name,item_description
0,102,2010,1/2 lb FlameThrower GrillBurger,"1/2 lb FlameThrower GrillBurger, Burgers"
1,10296,2010,"Chopped Farmhouse Chicken Salad, Crispy","Chopped Farmhouse Chicken Salad, Crispy, w/out..."
2,10297,2010,"Chopped Farmhouse Chicken Salad, Grilled","Chopped Farmhouse Chicken Salad, Grilled, w/ou..."
3,10298,2010,Chopped Side Salad,"Chopped Side Salad, Market Fresh Chopped Salads"
4,10299,2010,Philly Beef Toasted Subs,"Philly Beef Toasted Subs, Arby's Toasted Sandw..."
...,...,...,...,...
10357,977,2010,Egg & Cheese Breakfast Sandwich,"Egg & Cheese Breakfast Sandwich, Grilled Break..."
10358,978,2010,"Sausage, Egg & Cheese Breakfast Sandwich","Sausage, Egg & Cheese Breakfast Sandwich, Gril..."
10359,985,2010,"Breakfast Toaster, Bacon, Egg & Cheese","Breakfast Toaster, Bacon, Egg & Cheese, TOASTE..."
10360,987,2010,"Breakfast Toaster, Ham, Egg & Cheese","Breakfast Toaster, Ham, Egg & Cheese, TOASTER ..."


0
0
0
0
0
65
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,102,2010,24.0,3,1060.0,75.0,26.0,2.0,165.0,1980.0,NaN,41.0,54.0,9.0,2.0
1,10296,2010,2.0,8,460.0,25.0,9.0,0.0,65.0,1090.0,NaN,29.0,32.0,5.0,4.0
2,10297,2010,2.0,8,260.0,14.0,7.0,0.0,70.0,770.0,NaN,10.0,26.0,5.0,3.0
3,10298,2010,2.0,8,70.0,5.0,3.0,0.0,15.0,100.0,NaN,4.0,4.0,2.0,1.0
4,10299,2010,2.0,9,570.0,27.0,8.0,1.0,65.0,1490.0,NaN,55.0,29.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10357,977,2010,65.0,9,380.0,14.0,6.0,0.0,190.0,620.0,NaN,43.0,18.0,1.0,2.0
10358,978,2010,65.0,9,550.0,30.0,12.0,0.0,220.0,800.0,NaN,44.0,25.0,2.0,2.0
10359,985,2010,81.0,9,530.0,32.0,10.0,5.0,325.0,1440.0,NaN,40.0,20.0,7.0,2.0
10360,987,2010,81.0,9,490.0,26.0,8.0,5.0,325.0,1700.0,NaN,40.0,24.0,6.0,2.0


,combo_id,item_id,year,builds,cat_id
1,10241061,10296,2010,Main Item,8
2,10241062,10297,2010,Main Item,8
11,10231031,10357,2010,Main Item,5
12,10231032,10358,2010,Main Item,5
21,10231031,10392,2010,Accompanying Item,11
...,...,...,...,...,...
10266,223211601,86031,2010,Accompanying Item,11
10271,223311551,86035,2010,Accompanying Item,11
10272,223311552,86035,2010,Accompanying Item,11
10279,156656431,86042,2010,Main Item,9


In [137]:
dat = pd.read_csv('Customizable_Annual_Data/Annual_Data2008.csv')
transform_data(dat, year='2008')

,item_id,year,item_name,item_description
0,100,2008,"ButterBurger Cheese, Kids Meals","ButterBurger Cheese, Kids Meals, Kids"
1,100353,2008,Chicken Florentine w/ Dressing,"Chicken Florentine w/ Dressing, Insalata Salad..."
2,1009,2008,"Western w/ Cheese Breakfast Sandwich, 6""","Western w/ Cheese Breakfast Sandwich, Breakfas..."
3,101776,2008,"Vanilla Latte w/ Whole Milk, Grande","Vanilla Latte w/ Whole Milk, Grande"
4,102444,2008,"Guacamole, Side","Guacamole, Side, Specialties"
...,...,...,...,...
11673,989,2008,"Breakfast Toaster w/ Sausage, Egg & Cheese","Breakfast Toaster w/ Sausage, Egg & Cheese, Br..."
11674,99121,2008,Frontega Chicken on Focaccia,"Frontega Chicken on Focaccia, Hot Panini, From..."
11675,996,2008,Breakfast Sausage Bagel,"Breakfast Sausage Bagel, Breakfast"
11676,997,2008,Breakfast Bacon Bagel,"Breakfast Bacon Bagel, Breakfast"


0
0
0
0
0
85
0
0
0
0
0
0


,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,100,2008,23.0,3,396.0,20.0,8.0,1.0,78.0,955.0,NaN,36.0,22.0,NaN,1.0
1,100353,2008,76.0,8,840.0,53.0,8.0,NaN,NaN,5460.0,NaN,62.0,33.0,NaN,7.0
2,1009,2008,84.0,9,450.0,19.0,8.0,0.0,200.0,1390.0,NaN,46.0,28.0,6.0,5.0
3,101776,2008,82.0,2,280.0,11.0,6.0,0.0,35.0,135.0,NaN,36.0,11.0,33.0,0.0
4,102444,2008,86.0,11,70.0,5.0,1.0,0.0,0.0,180.0,NaN,5.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11673,989,2008,81.0,9,620.0,42.0,13.0,1.0,340.0,1380.0,NaN,40.0,20.0,6.0,2.0
11674,99121,2008,65.0,9,810.0,34.0,9.0,5.0,100.0,2140.0,NaN,79.0,45.0,6.0,5.0
11675,996,2008,83.0,9,576.0,295.0,115.0,0.0,212.0,1120.0,NaN,51.0,26.0,6.0,2.0
11676,997,2008,83.0,9,458.0,165.0,7.0,0.0,193.0,1102.0,NaN,51.0,25.0,7.0,2.0


,combo_id,item_id,year,builds,cat_id
124,103871011,10508,2008,Accompanying Item,11
125,1038710110,10508,2008,Accompanying Item,11
126,1038710111,10508,2008,Accompanying Item,11
127,1038710112,10508,2008,Accompanying Item,11
128,1038710113,10508,2008,Accompanying Item,11
...,...,...,...,...,...
11556,16822481,86031,2008,Accompanying Item,11
11590,1038710163,90970,2008,Main Item,4
11591,1038710164,90972,2008,Main Item,4
11597,108181102,91453,2008,Main Item,5


### Note:

We have a small amount of NULLs in restaurant id column in year 2008, 2010, and 2012
which means that these restaurants were no longer in the menu data record in later years.
So they are ok to ignore.